# ES → BBF Service Delivery Migration Investigation

This notebook investigates ES Order/OrderItem data to prepare for migration to BBF Service__c and Service_Charge__c.

## Investigation Goals
1. **Order Status Distribution** - Count Orders by Status to understand migration scope
2. **Data Quality Validation** - Verify required fields are populated
3. **Relationship Integrity** - Check lookups to BAN, Location, Node, Account
4. **OrderItem Analysis** - Understand charge structure and amounts
5. **Gap Identification** - Find data issues before migration

## Active Status Filter (for Service__c)
```python
ACTIVE_STATUSES = [
    'Activated',
    'Suspended (Late Payment)',
    'Disconnect in Progress'
]
```

## Target Mapping
- ES Order (active) → BBF Service__c
- ES OrderItem (for active Orders) → BBF Service_Charge__c

---
**Created:** December 11, 2024  
**Project:** ES → BBF Salesforce Migration

In [2]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [3]:
# === CONFIGURATION ===

# ES (Source) Credentials
ES_USERNAME = "vlettau@everstream.net.uat"
ES_PASSWORD = "MNlkpo0987)(*&"
ES_TOKEN = "nSBoNS97wYLCRW2JP2JARR12"
ES_DOMAIN = "test"  # or 'login' for production


# Active Status Filter for Service__c migration
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# Output settings
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"es_service_delivery_investigation_{TIMESTAMP}.xlsx"

print("📋 Configuration loaded")
print(f"   Active Statuses: {ACTIVE_STATUSES}")
print(f"   Output: {OUTPUT_FILE}")

📋 Configuration loaded
   Active Statuses: ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
   Output: es_service_delivery_investigation_20251211_181011.xlsx


In [4]:
# === CONNECT TO ES SALESFORCE ===

print("🔌 Connecting to ES Salesforce...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

🔌 Connecting to ES Salesforce...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com


---
## Section 1: Order Status Distribution

Understanding the breakdown of Orders by Status to scope the migration.

In [5]:
# === 1.1 COUNT ORDERS BY STATUS ===

print("📊 Querying Order Status distribution...")

status_query = """
SELECT Status, COUNT(Id) total
FROM Order
GROUP BY Status
ORDER BY COUNT(Id) DESC
"""

status_result = es_sf.query_all(status_query)
status_df = pd.DataFrame(status_result["records"]).drop(
    columns=["attributes"], errors="ignore"
)

# Add migration flag
status_df["Include_In_Migration"] = status_df["Status"].isin(ACTIVE_STATUSES)

print("\n=== ES Order Status Distribution ===")
print(status_df.to_string(index=False))

# Summary
total_orders = status_df["total"].sum()
active_orders = status_df[status_df["Include_In_Migration"]]["total"].sum()

print(f"\n📈 Summary:")
print(f"   Total Orders: {total_orders:,}")
print(f"   Active Orders (to migrate): {active_orders:,}")
print(f"   Percentage: {active_orders/total_orders*100:.1f}%")

📊 Querying Order Status distribution...

=== ES Order Status Distribution ===
                  Status  total  Include_In_Migration
            Disconnected  21432                 False
               Activated  17819                  True
               Cancelled   8117                 False
             Deactivated   3247                 False
    In Progress (future)    910                 False
                   Draft    837                 False
  Disconnect in Progress    364                  True
Suspended (Late Payment)      1                  True

📈 Summary:
   Total Orders: 52,727
   Active Orders (to migrate): 18,184
   Percentage: 34.5%


In [6]:
# === 1.2 BREAKDOWN OF ACTIVE STATUSES ===

print("\n=== Active Status Breakdown (for Service__c) ===")
active_df = status_df[status_df["Include_In_Migration"]].copy()
print(active_df.to_string(index=False))

# Store for later use
ACTIVE_ORDER_COUNT = active_orders
print(f"\n✅ {ACTIVE_ORDER_COUNT:,} Orders will become Service__c records")


=== Active Status Breakdown (for Service__c) ===
                  Status  total  Include_In_Migration
               Activated  17819                  True
  Disconnect in Progress    364                  True
Suspended (Late Payment)      1                  True

✅ 18,184 Orders will become Service__c records


---
## Section 2: Data Quality Validation

Checking that required fields are populated on active Orders.

In [7]:
# === 2.1 CRITICAL FIELD: Billing_Invoice__c (Required for BBF Service__c) ===

print("🔍 Checking Billing_Invoice__c (CRITICAL - required for BBF Service__c)...")

billing_check_query = """
SELECT 
    Status,
    COUNT(Id) total,
    SUM(CASE WHEN Billing_Invoice__c = null THEN 1 ELSE 0 END) missing_billing
FROM Order
WHERE Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
GROUP BY Status
"""

# Note: SOQL doesn't support CASE/SUM like this, so we'll do it differently
# Query Orders with missing Billing_Invoice__c

missing_billing_query = """
SELECT Status, COUNT(Id) missing_count
FROM Order
WHERE Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
AND Billing_Invoice__c = null
GROUP BY Status
"""

try:
    missing_result = es_sf.query_all(missing_billing_query)
    if missing_result["totalSize"] > 0:
        missing_df = pd.DataFrame(missing_result["records"]).drop(
            columns=["attributes"], errors="ignore"
        )
        print("\n⚠️ WARNING: Orders MISSING Billing_Invoice__c:")
        print(missing_df.to_string(index=False))
        print("\n❌ These Orders cannot be migrated without a BAN!")
    else:
        print("\n✅ All active Orders have Billing_Invoice__c populated")
except Exception as e:
    print(f"Query error: {e}")
    print("Running alternative check...")

🔍 Checking Billing_Invoice__c (CRITICAL - required for BBF Service__c)...

⚠️ WARNING: Orders MISSING Billing_Invoice__c:
                Status  missing_count
             Activated           3433
Disconnect in Progress             18

❌ These Orders cannot be migrated without a BAN!


In [8]:
# === 2.2 CHECK ALL KEY LOOKUP FIELDS ===

print("🔍 Checking key lookup fields on active Orders...")

# Build status filter
status_filter = "','".join(ACTIVE_STATUSES)

# Query to count nulls for each important field
fields_to_check = [
    ("Billing_Invoice__c", "BAN (CRITICAL)"),
    ("AccountId", "Account"),
    ("Address_A__c", "A Location"),
    ("Address_Z__c", "Z Location"),
    ("Node__c", "Node"),
    ("Service_ID__c", "Service ID"),
    ("Service_Start_Date__c", "Service Start Date"),
    ("OpportunityId", "Opportunity"),
]

print("\n=== Field Population Analysis (Active Orders) ===")
print(f"{'Field':<30} {'Populated':>12} {'Missing':>12} {'% Complete':>12}")
print("-" * 70)

field_stats = []
for field_api, field_label in fields_to_check:
    # Count where field is NOT null
    populated_query = f"""
    SELECT COUNT(Id) cnt
    FROM Order
    WHERE Status IN ('{status_filter}')
    AND {field_api} != null
    """

    try:
        result = es_sf.query(populated_query)
        populated = result["records"][0]["cnt"]
        missing = ACTIVE_ORDER_COUNT - populated
        pct = (populated / ACTIVE_ORDER_COUNT * 100) if ACTIVE_ORDER_COUNT > 0 else 0

        status = "✅" if pct == 100 else "⚠️" if pct >= 90 else "❌"
        print(
            f"{status} {field_label:<28} {populated:>12,} {missing:>12,} {pct:>11.1f}%"
        )

        field_stats.append(
            {
                "Field": field_label,
                "API Name": field_api,
                "Populated": populated,
                "Missing": missing,
                "Percent Complete": pct,
            }
        )
    except Exception as e:
        print(f"❓ {field_label:<28} Error: {e}")

field_stats_df = pd.DataFrame(field_stats)

🔍 Checking key lookup fields on active Orders...

=== Field Population Analysis (Active Orders) ===
Field                             Populated      Missing   % Complete
----------------------------------------------------------------------
❌ BAN (CRITICAL)                     14,733        3,451        81.0%
✅ Account                            18,184            0       100.0%
⚠️ A Location                         17,983          201        98.9%
✅ Z Location                         18,184            0       100.0%
❌ Node                                2,373       15,811        13.0%
✅ Service ID                         18,184            0       100.0%
❌ Service Start Date                  9,309        8,875        51.2%
❌ Opportunity                        10,029        8,155        55.2%


In [9]:
# === 2.3 SAMPLE ORDERS MISSING CRITICAL DATA ===

print("🔍 Finding sample Orders with missing critical data...")

# Query Orders missing Billing_Invoice__c (most critical)
missing_critical_query = f"""
SELECT Id, Name, Service_ID__c, Status, AccountId, Billing_Invoice__c,
       Address_A__c, Address_Z__c, Node__c, Service_Start_Date__c
FROM Order
WHERE Status IN ('{status_filter}')
AND (Billing_Invoice__c = null OR AccountId = null)
LIMIT 20
"""

try:
    missing_result = es_sf.query(missing_critical_query)
    if missing_result["totalSize"] > 0:
        missing_df = pd.DataFrame(missing_result["records"]).drop(
            columns=["attributes"], errors="ignore"
        )
        print(f"\n⚠️ Found {missing_result['totalSize']} Orders missing critical data:")
        print(missing_df.to_string())
    else:
        print("\n✅ No Orders missing critical data (Billing_Invoice__c, AccountId)")
except Exception as e:
    print(f"Query error: {e}")

🔍 Finding sample Orders with missing critical data...

⚠️ Found 20 Orders missing critical data:
                    Id  Name               Service_ID__c     Status           AccountId Billing_Invoice__c        Address_A__c        Address_Z__c Node__c Service_Start_Date__c
0   8014P000002PU30QAG  None           EVM-ETHS-ON-20794  Activated  0010B00001jOrP8QAK               None  aD80B0000004DMxSAM  aD80B0000004I0USAU    None            2020-02-28
1   8010B0000005vofQAA  None            EV-ETHS-ON-03801  Activated  0010B00001laHGSQA2               None  aD80B0000004DB3SAM  aD80B0000004DIoSAM    None            2017-01-18
2   8010B000000uH1rQAE  None            EV-ETHS-ON-12313  Activated  0010B00001lbKt2QAE               None  aD84P000000blcLSAQ  aD80B0000004D14SAE    None            2017-11-17
3   801Rn00000QQgUUIA1  None         EV-ETHS-ON-139099 b  Activated  0010B00001mAVIiQAO               None  aD84P000000XfNCSA0  aD83g0000005ZyWCAU    None                  None
4   801Rn00000XydO

---
## Section 3: OrderItem Analysis

Understanding the OrderItem structure for Service_Charge__c creation.

In [10]:
# === 3.1 COUNT ORDERITEMS FOR ACTIVE ORDERS ===

print("📊 Counting OrderItems for active Orders...")

orderitem_count_query = f"""
SELECT COUNT(Id) total
FROM OrderItem
WHERE Order.Status IN ('{status_filter}')
"""

try:
    oi_result = es_sf.query(orderitem_count_query)
    total_orderitems = oi_result["records"][0]["total"]
    print(f"\n✅ Total OrderItems for active Orders: {total_orderitems:,}")
    print(f"   Average OrderItems per Order: {total_orderitems/ACTIVE_ORDER_COUNT:.1f}")
    TOTAL_ORDERITEMS = total_orderitems
except Exception as e:
    print(f"Query error: {e}")
    TOTAL_ORDERITEMS = 0

📊 Counting OrderItems for active Orders...

✅ Total OrderItems for active Orders: 28,979
   Average OrderItems per Order: 1.6


In [11]:
# === 3.2 CHARGE TYPE DISTRIBUTION ===

print("📊 Analyzing Charge Type distribution (SBQQ__ChargeType__c)...")

charge_type_query = f"""
SELECT SBQQ__ChargeType__c, COUNT(Id) total
FROM OrderItem
WHERE Order.Status IN ('{status_filter}')
GROUP BY SBQQ__ChargeType__c
ORDER BY COUNT(Id) DESC
"""

try:
    charge_result = es_sf.query_all(charge_type_query)
    charge_df = pd.DataFrame(charge_result["records"]).drop(
        columns=["attributes"], errors="ignore"
    )

    # Map to BBF Service_Type_Charge__c
    charge_type_map = {
        "Recurring": "MRC",
        "One-Time": "NRC",
        "Usage": "Usage",
        None: "(blank)",
    }
    charge_df["BBF_Charge_Type"] = charge_df["SBQQ__ChargeType__c"].map(
        lambda x: charge_type_map.get(x, "Unknown")
    )

    print("\n=== Charge Type Distribution ===")
    print(charge_df.to_string(index=False))

except Exception as e:
    print(f"Query error: {e}")

📊 Analyzing Charge Type distribution (SBQQ__ChargeType__c)...

=== Charge Type Distribution ===
SBQQ__ChargeType__c  total BBF_Charge_Type
               None  28979         (blank)


In [12]:
# === 3.3 MRC/NRC AMOUNT SUMMARY ===

print("💰 Analyzing MRC/NRC amounts...")

# Sample OrderItems to check amounts
amount_sample_query = f"""
SELECT 
    Id, OrderId, Product2Id, SBQQ__ChargeType__c,
    Total_MRC_Amortized__c, 
    NRC_IRU_FEE__c,
    NRC_Non_Amortized__c,
    Total_NRC_Amortized__c,
    Vendor_NRC__c,
    Product_Family__c,
    Product_Name__c
FROM OrderItem
WHERE Order.Status IN ('{status_filter}')
AND Total_MRC_Amortized__c > 0
LIMIT 50
"""

try:
    amount_result = es_sf.query(amount_sample_query)
    amount_df = pd.DataFrame(amount_result["records"]).drop(
        columns=["attributes"], errors="ignore"
    )

    print(f"\n=== Sample OrderItems with MRC > 0 ({len(amount_df)} records) ===")
    print(amount_df.head(10).to_string())

    # Summary stats
    print("\n=== MRC Statistics ===")
    print(amount_df["Total_MRC_Amortized__c"].describe())

except Exception as e:
    print(f"Query error: {e}")

💰 Analyzing MRC/NRC amounts...

=== Sample OrderItems with MRC > 0 (50 records) ===
                   Id             OrderId          Product2Id SBQQ__ChargeType__c  Total_MRC_Amortized__c  NRC_IRU_FEE__c  NRC_Non_Amortized__c  Total_NRC_Amortized__c  Vendor_NRC__c                 Product_Family__c              Product_Name__c
0  802VA00000JYH28YAH  8010B0000002fQYQAY  01tU0000002M0dDIAS                None                   280.0             0.0                   0.0                     NaN            NaN               IPV.4 Address Space  IPV.4 (28 Public-facing IP)
1  8020B000001dvE7QAI  8010B0000002geCQAQ  01t0B0000075Y0zQAE                None                  1750.0             NaN                   0.0                    0.00            NaN  Dedicated Internet Access (DIAS)                     Internet
2  8020B000001e4tQQAQ  8010B0000002pBsQAI  01t0B0000075Y0zQAE                None                   800.0             0.0                   0.0                    0.00           

In [13]:
# === 3.4 PRODUCT FAMILY DISTRIBUTION ===

print("📊 Analyzing Product Family distribution (for Product_Simple__c mapping)...")

product_query = f"""
SELECT Product_Family__c, COUNT(Id) total
FROM OrderItem
WHERE Order.Status IN ('{status_filter}')
GROUP BY Product_Family__c
ORDER BY COUNT(Id) DESC
"""

try:
    product_result = es_sf.query_all(product_query)
    product_df = pd.DataFrame(product_result["records"]).drop(
        columns=["attributes"], errors="ignore"
    )

    print("\n=== Product Family Distribution ===")
    print(product_df.to_string(index=False))
    print(
        f"\n📋 {len(product_df)} unique Product Families to map to BBF Product_Simple__c"
    )

except Exception as e:
    print(f"Query error: {e}")

📊 Analyzing Product Family distribution (for Product_Simple__c mapping)...
Query error: Malformed request https://everstream--uat.sandbox.my.salesforce.com/services/data/v59.0/query/?q=%0ASELECT+Product_Family__c%2C+COUNT%28Id%29+total%0AFROM+OrderItem%0AWHERE+Order.Status+IN+%28%27Activated%27%2C%27Suspended+%28Late+Payment%29%27%2C%27Disconnect+in+Progress%27%29%0AGROUP+BY+Product_Family__c%0AORDER+BY+COUNT%28Id%29+DESC%0A. Response content: [{'message': "\nGROUP BY Product_Family__c\n         ^\nERROR at Row:4:Column:10\nfield 'Product_Family__c' can not be grouped in a query call", 'errorCode': 'MALFORMED_QUERY'}]


---
## Section 4: Relationship Integrity

Verifying that lookup relationships are valid for migration.

In [14]:
# === 4.1 VERIFY BILLING_INVOICE (BAN) RECORDS EXIST ===

print("🔗 Verifying Billing_Invoice__c records exist for active Orders...")

# Get distinct Billing_Invoice__c IDs from active Orders
ban_ids_query = f"""
SELECT Billing_Invoice__c, COUNT(Id) order_count
FROM Order
WHERE Status IN ('{status_filter}')
AND Billing_Invoice__c != null
GROUP BY Billing_Invoice__c
"""

try:
    ban_result = es_sf.query_all(ban_ids_query)
    ban_df = pd.DataFrame(ban_result["records"]).drop(
        columns=["attributes"], errors="ignore"
    )

    unique_bans = len(ban_df)
    print(
        f"\n✅ Found {unique_bans:,} unique Billing_Invoice__c records referenced by active Orders"
    )

    # Show distribution
    print("\n=== Orders per BAN (Top 10) ===")
    print(ban_df.nlargest(10, "order_count").to_string(index=False))

except Exception as e:
    print(f"Query error: {e}")

🔗 Verifying Billing_Invoice__c records exist for active Orders...
Query error: Malformed request https://everstream--uat.sandbox.my.salesforce.com/services/data/v59.0/query/?q=%0ASELECT+Billing_Invoice__c%2C+COUNT%28Id%29+order_count%0AFROM+Order%0AWHERE+Status+IN+%28%27Activated%27%2C%27Suspended+%28Late+Payment%29%27%2C%27Disconnect+in+Progress%27%29%0AAND+Billing_Invoice__c+%21%3D+null%0AGROUP+BY+Billing_Invoice__c%0A. Response content: [{'message': 'Aggregate query does not support queryMore(), use LIMIT to restrict the results to a single batch', 'errorCode': 'EXCEEDED_ID_LIMIT'}]


In [15]:
# === 4.2 VERIFY ADDRESS (LOCATION) RECORDS ===

print("🔗 Verifying Address__c records for A/Z locations...")

# Count distinct addresses
address_a_query = f"""
SELECT COUNT(Id) total
FROM Order
WHERE Status IN ('{status_filter}')
AND Address_A__c != null
"""

address_z_query = f"""
SELECT COUNT(Id) total
FROM Order
WHERE Status IN ('{status_filter}')
AND Address_Z__c != null
"""

try:
    a_result = es_sf.query(address_a_query)
    z_result = es_sf.query(address_z_query)

    a_count = a_result["records"][0]["total"]
    z_count = z_result["records"][0]["total"]

    print(f"\n=== Location Population ===")
    print(
        f"   Address_A__c populated: {a_count:,} ({a_count/ACTIVE_ORDER_COUNT*100:.1f}%)"
    )
    print(
        f"   Address_Z__c populated: {z_count:,} ({z_count/ACTIVE_ORDER_COUNT*100:.1f}%)"
    )

except Exception as e:
    print(f"Query error: {e}")

🔗 Verifying Address__c records for A/Z locations...

=== Location Population ===
   Address_A__c populated: 17,983 (98.9%)
   Address_Z__c populated: 18,184 (100.0%)


In [16]:
# === 4.3 VERIFY NODE RECORDS ===

print("🔗 Verifying Node__c records...")

node_query = f"""
SELECT Node__c, COUNT(Id) order_count
FROM Order
WHERE Status IN ('{status_filter}')
AND Node__c != null
GROUP BY Node__c
"""

try:
    node_result = es_sf.query_all(node_query)
    node_df = pd.DataFrame(node_result["records"]).drop(
        columns=["attributes"], errors="ignore"
    )

    unique_nodes = len(node_df)
    orders_with_node = node_df["order_count"].sum()

    print(f"\n=== Node Population ===")
    print(f"   Unique Nodes: {unique_nodes:,}")
    print(
        f"   Orders with Node: {orders_with_node:,} ({orders_with_node/ACTIVE_ORDER_COUNT*100:.1f}%)"
    )
    print(f"\n   ⚠️ Note: ES has single Node__c, BBF needs A_Node__c + Z_Node__c")
    print(
        f"   Recommendation: Map Node__c → A_Node__c, leave Z_Node__c for post-migration"
    )

except Exception as e:
    print(f"Query error: {e}")

🔗 Verifying Node__c records...
Query error: Malformed request https://everstream--uat.sandbox.my.salesforce.com/services/data/v59.0/query/?q=%0ASELECT+Node__c%2C+COUNT%28Id%29+order_count%0AFROM+Order%0AWHERE+Status+IN+%28%27Activated%27%2C%27Suspended+%28Late+Payment%29%27%2C%27Disconnect+in+Progress%27%29%0AAND+Node__c+%21%3D+null%0AGROUP+BY+Node__c%0A. Response content: [{'message': 'Aggregate query does not support queryMore(), use LIMIT to restrict the results to a single batch', 'errorCode': 'EXCEEDED_ID_LIMIT'}]


---
## Section 5: Sample Data Review

Pulling sample records to verify field values before migration.

In [17]:
# === 5.1 SAMPLE ACTIVATED ORDERS ===

print("📋 Pulling sample Activated Orders...")

sample_order_query = """
SELECT 
    Id, Name, Service_ID__c, Status,
    AccountId, Account.Name,
    Billing_Invoice__c,
    Address_A__c, Address_Z__c, Node__c,
    Service_Start_Date__c, Service_End_Date__c,
    Service_Provided__c,
    SOF_MRC__c,
    OSS_Service_ID__c,
    Vendor_Circuit_ID__c
FROM Order
WHERE Status = 'Activated'
LIMIT 20
"""

try:
    sample_result = es_sf.query(sample_order_query)
    sample_df = pd.DataFrame(sample_result["records"])

    # Flatten Account.Name
    if "Account" in sample_df.columns:
        sample_df["Account_Name"] = sample_df["Account"].apply(
            lambda x: x.get("Name") if isinstance(x, dict) else None
        )
        sample_df = sample_df.drop(columns=["Account", "attributes"], errors="ignore")

    print(f"\n=== Sample Activated Orders ({len(sample_df)} records) ===")
    print(sample_df.to_string())

    # Store for export
    sample_orders_df = sample_df

except Exception as e:
    print(f"Query error: {e}")

📋 Pulling sample Activated Orders...

=== Sample Activated Orders (20 records) ===
                    Id  Name         Service_ID__c     Status           AccountId  Billing_Invoice__c        Address_A__c        Address_Z__c Node__c Service_Start_Date__c Service_End_Date__c  Service_Provided__c  SOF_MRC__c     OSS_Service_ID__c Vendor_Circuit_ID__c                                           Account_Name
0   8010B0000002fQTQAY  None      EV-ETHS-ON-12424  Activated  001U000001UUy0ZIAT                None  aD84P000000XfNCSA0  aD84P000000XenTSAS    None            2017-10-06                None               1000.0         0.0      EV-ETHS-ON-12424                 None  INTERNAL EVERSTREAM NETWORK RESOURCES (DO NOT DELETE)
1   8010B0000002fQYQAY  None      EV-ETHc-ON-12425  Activated  001U000001UUy0ZIAT                None                None  aD80B0000004CnoSAE    None            2019-08-01                None                  0.0       280.0      EV-ETHc-ON-12425                 None  IN

In [18]:
# === 5.2 SAMPLE ORDERITEMS FOR ACTIVATED ORDERS ===

print("📋 Pulling sample OrderItems...")

sample_oi_query = """
SELECT 
    Id, OrderId, Order.Service_ID__c,
    Product2Id, Product_Family__c, Product_Name__c,
    SBQQ__ChargeType__c,
    Total_MRC_Amortized__c,
    NRC_IRU_FEE__c, NRC_Non_Amortized__c, Total_NRC_Amortized__c,
    Bandwidth_NEW__c,
    Product_Service_Term__c
FROM OrderItem
WHERE Order.Status = 'Activated'
AND Total_MRC_Amortized__c > 0
LIMIT 30
"""

try:
    sample_oi_result = es_sf.query(sample_oi_query)
    sample_oi_df = pd.DataFrame(sample_oi_result["records"])

    # Flatten Order.Service_ID__c
    if "Order" in sample_oi_df.columns:
        sample_oi_df["Order_Service_ID"] = sample_oi_df["Order"].apply(
            lambda x: x.get("Service_ID__c") if isinstance(x, dict) else None
        )
        sample_oi_df = sample_oi_df.drop(
            columns=["Order", "attributes"], errors="ignore"
        )

    print(f"\n=== Sample OrderItems ({len(sample_oi_df)} records) ===")
    print(sample_oi_df.to_string())

    # Store for export
    sample_orderitems_df = sample_oi_df

except Exception as e:
    print(f"Query error: {e}")

📋 Pulling sample OrderItems...

=== Sample OrderItems (30 records) ===
                    Id             OrderId          Product2Id                 Product_Family__c              Product_Name__c SBQQ__ChargeType__c  Total_MRC_Amortized__c  NRC_IRU_FEE__c  NRC_Non_Amortized__c  Total_NRC_Amortized__c Bandwidth_NEW__c Product_Service_Term__c    Order_Service_ID
0   802VA00000JYH28YAH  8010B0000002fQYQAY  01tU0000002M0dDIAS               IPV.4 Address Space  IPV.4 (28 Public-facing IP)                None                   280.0             0.0                   0.0                     NaN             None                    None    EV-ETHc-ON-12425
1   8020B000001dvE7QAI  8010B0000002geCQAQ  01t0B0000075Y0zQAE  Dedicated Internet Access (DIAS)                     Internet                None                  1750.0             NaN                   0.0                    0.00              500               36 Months    EV-ETHS-ON-12436
2   8020B000001e4tQQAQ  8010B0000002pBsQAI  01t0B0

---
## Section 6: Export Results

Save investigation results to Excel for review.

In [ ]:
# === 6.1 EXPORT TO EXCEL ===

print(f"📁 Exporting results to {OUTPUT_FILE}...")

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:

    # Summary sheet
    summary_data = {
        "Metric": [
            "Total Orders (all statuses)",
            "Active Orders (to migrate)",
            "Total OrderItems (for active)",
            "Avg OrderItems per Order",
            "Investigation Date",
        ],
        "Value": [
            f"{total_orders:,}",
            f"{ACTIVE_ORDER_COUNT:,}",
            f"{TOTAL_ORDERITEMS:,}" if TOTAL_ORDERITEMS else "N/A",
            (
                f"{TOTAL_ORDERITEMS/ACTIVE_ORDER_COUNT:.1f}"
                if TOTAL_ORDERITEMS and ACTIVE_ORDER_COUNT
                else "N/A"
            ),
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        ],
    }
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name="Summary", index=False)

    # Status distribution
    status_df.to_excel(writer, sheet_name="Order_Status", index=False)

    # Field population stats
    if "field_stats_df" in dir() and not field_stats_df.empty:
        field_stats_df.to_excel(writer, sheet_name="Field_Population", index=False)

    # Charge type distribution
    if "charge_df" in dir() and not charge_df.empty:
        charge_df.to_excel(writer, sheet_name="Charge_Types", index=False)

    # Product family distribution
    if "product_df" in dir() and not product_df.empty:
        product_df.to_excel(writer, sheet_name="Product_Families", index=False)

    # Sample Orders
    if "sample_orders_df" in dir() and not sample_orders_df.empty:
        sample_orders_df.to_excel(writer, sheet_name="Sample_Orders", index=False)

    # Sample OrderItems
    if "sample_orderitems_df" in dir() and not sample_orderitems_df.empty:
        sample_orderitems_df.to_excel(
            writer, sheet_name="Sample_OrderItems", index=False
        )

print(f"\n✅ Investigation results exported to: {OUTPUT_FILE}")

---
## Section 7: Investigation Summary

Final summary and next steps.

In [ ]:
# === 7.1 FINAL SUMMARY ===

print("=" * 70)
print("ES SERVICE DELIVERY INVESTIGATION - SUMMARY")
print("=" * 70)

print(
    f"""
📊 MIGRATION SCOPE
   Active Orders to migrate:     {ACTIVE_ORDER_COUNT:,}
   OrderItems to migrate:        {TOTAL_ORDERITEMS:,} (approx)
   
   ➡️  Will create {ACTIVE_ORDER_COUNT:,} BBF Service__c records
   ➡️  Will create ~{TOTAL_ORDERITEMS:,} BBF Service_Charge__c records

📋 ACTIVE STATUS FILTER
   ✅ Activated
   ✅ Suspended (Late Payment)
   ✅ Disconnect in Progress

⚠️  DATA QUALITY ISSUES TO ADDRESS
   (Review Field_Population sheet in Excel output)

🔗 DEPENDENCY MAPPING
   ES Billing_Invoice__c → BBF BAN__c (CRITICAL - must exist)
   ES Address_A__c → BBF Location__c → Service__c.A_Location__c
   ES Address_Z__c → BBF Location__c → Service__c.Z_Location__c
   ES Node__c → BBF Node__c → Service__c.A_Node__c
   ES AccountId → BBF Account → Service__c.Account__c

📁 OUTPUT FILE: {OUTPUT_FILE}
"""
)

print("=" * 70)
print("NEXT STEPS")
print("=" * 70)
print(
    """
1. Review Excel output for data quality issues
2. Resolve any Orders missing Billing_Invoice__c
3. Create Product_Simple__c mapping (ES Product Family → BBF picklist)
4. Verify prerequisite migrations complete:
   - BAN__c (from Billing_Invoice__c)
   - Location__c (from Address__c)
   - Node__c
   - Account
5. Create Service__c migration notebook
6. Create Service_Charge__c migration notebook
"""
)

---
## Section 8: Additional Investigation (Add as needed)

Space for additional queries as issues arise during migration planning.

In [ ]:
# === 8.1 PLACEHOLDER FOR ADDITIONAL QUERIES ===

# Add custom queries here as needed during investigation
# Example:
# custom_query = """
# SELECT ...
# FROM Order
# WHERE ...
# """
# result = es_sf.query(custom_query)
# print(pd.DataFrame(result['records']))

In [ ]:
# === 8.2 PLACEHOLDER ===

pass

In [ ]:
# === 8.3 PLACEHOLDER ===

pass